
<br>
================================================================<br>
Use rolling-ball algorithm for estimating background intensity<br>
================================================================<br>
The rolling-ball algorithm estimates the background intensity of a grayscale<br>
image in case of uneven exposure. It is frequently used in biomedical<br>
image processing and was first proposed by Stanley R. Sternberg in<br>
1983 [1]_.<br>
The algorithm works as a filter and is quite intuitive. We think of the image<br>
as a surface that has unit-sized blocks stacked on top of each other in place<br>
of each pixel. The number of blocks, and hence surface height, is determined<br>
by the intensity of the pixel. To get the intensity of the background at a<br>
desired (pixel) position, we imagine submerging a ball under the surface at the<br>
desired position. Once it is completely covered by the blocks, the apex of<br>
the ball determines the intensity of the background at that position. We can<br>
then *roll* this ball around below the surface to get the background values for<br>
the entire image.<br>
Scikit-image implements a general version of this rolling-ball algorithm, which<br>
allows you to not just use balls, but arbitrary shapes as kernel and works on<br>
n-dimensional ndimages. This allows you to directly filter RGB images or filter<br>
image stacks along any (or all) spacial dimensions.<br>
.. [1] Sternberg, Stanley R. "Biomedical image processing." Computer 1 (1983):<br>
    22-34. :DOI:`10.1109/MC.1983.1654163`<br>
Classic rolling ball<br>
-------------------------------<br>
In scikit-image, the rolling ball algorithm assumes that your background has<br>
low intensity (black), whereas the features have high intensity (white). If<br>
this is the case for your image, you can directly use the filter like so:<br>


In [ ]:
import imageio
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from skimage import (
    data, restoration, util
)

In [ ]:
def plot_result(image, background):
    fig, ax = plt.subplots(nrows=1, ncols=3)
    ax[0].imshow(image, cmap='gray')
    ax[0].set_title('Original image')
    ax[0].axis('off')
    ax[1].imshow(background, cmap='gray')
    ax[1].set_title('Background')
    ax[1].axis('off')
    ax[2].imshow(image - background, cmap='gray')
    ax[2].set_title('Result')
    ax[2].axis('off')
    fig.tight_layout()

In [ ]:
image = data.coins()

In [ ]:
background = restoration.rolling_ball(image)

In [ ]:
plot_result(image, background)
plt.show()

####################################################################<br>
White background<br>
----------------<br>
<br>
If you have dark features on a bright background, you need to invert<br>
the image before you pass it into the algorithm, and then invert the<br>
result. This can be accomplished via:

In [ ]:
image = data.page()
image_inverted = util.invert(image)

In [ ]:
background_inverted = restoration.rolling_ball(image_inverted, radius=45)
filtered_image_inverted = image_inverted - background_inverted
filtered_image = util.invert(filtered_image_inverted)
background = util.invert(background_inverted)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3)

In [ ]:
ax[0].imshow(image, cmap='gray')
ax[0].set_title('Original image')
ax[0].axis('off')

In [ ]:
ax[1].imshow(background, cmap='gray')
ax[1].set_title('Background')
ax[1].axis('off')

In [ ]:
ax[2].imshow(filtered_image, cmap='gray')
ax[2].set_title('Result')
ax[2].axis('off')

In [ ]:
fig.tight_layout()

In [ ]:
plt.show()

####################################################################<br>
Be careful not to fall victim to an integer underflow when subtracting<br>
a bright background. For example, this code looks correct, but may<br>
suffer from an underflow leading to unwanted artifacts. You can see<br>
this in the top right corner of the visualization.

In [ ]:
image = data.page()
image_inverted = util.invert(image)

In [ ]:
background_inverted = restoration.rolling_ball(image_inverted, radius=45)
background = util.invert(background_inverted)
underflow_image = image - background  # integer underflow occurs here

correct subtraction

In [ ]:
correct_image = util.invert(image_inverted - background_inverted)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2)

In [ ]:
ax[0].imshow(underflow_image, cmap='gray')
ax[0].set_title('Background Removal with Underflow')
ax[0].axis('off')

In [ ]:
ax[1].imshow(correct_image, cmap='gray')
ax[1].set_title('Correct Background Removal')
ax[1].axis('off')

In [ ]:
fig.tight_layout()

In [ ]:
plt.show()

####################################################################<br>
Image Datatypes<br>
---------------<br>
<br>
``rolling_ball`` can handle datatypes other than `np.uint8`. You can<br>
pass them into the function in the same way.

In [ ]:
image = data.coins()[:200, :200].astype(np.uint16)

In [ ]:
background = restoration.rolling_ball(image, radius=70.5)
plot_result(image, background)
plt.show()

####################################################################<br>
However, you need to be careful if you use floating point images<br>
that have been normalized to ``[0, 1]``. In this case the ball will<br>
be much larger than the image intensity, which can lead to<br>
unexpected results.

In [ ]:
image = util.img_as_float(data.coins()[:200, :200])

In [ ]:
background = restoration.rolling_ball(image, radius=70.5)
plot_result(image, background)
plt.show()

####################################################################<br>
Because ``radius=70.5`` is much larger than the maximum intensity of<br>
the image, the effective kernel size is reduced significantly, i.e.,<br>
only a small cap (approximately ``radius=10``) of the ball is rolled<br>
around in the image. You can find a reproduction of this strange<br>
effect in the ``Advanced Shapes`` section below.<br>
<br>
To get the expected result, you need to reduce the intensity of the<br>
kernel. This is done by specifying the kernel manually using the<br>
``kernel`` argument.<br>
<br>
Note: The radius is equal to the length of a semi-axis of an<br>
ellipsis, which is *half* a full axis. Hence, the kernel shape is<br>
multipled by two.

In [ ]:
normalized_radius = 70.5 / 255
image = util.img_as_float(data.coins())
kernel = restoration.ellipsoid_kernel(
    (70.5 * 2, 70.5 * 2),
    normalized_radius * 2
)

In [ ]:
background = restoration.rolling_ball(
    image,
    kernel=kernel
)
plot_result(image, background)
plt.show()

####################################################################<br>
Advanced Shapes<br>
-----------------<br>
<br>
By default, ``rolling_ball`` uses a ball shaped kernel (surprise).<br>
Sometimes, this can be too limiting - as in the example above -,<br>
because the intensity dimension has a different scale compared to<br>
the spatial dimensions, or because the image dimensions may have<br>
different meanings - one could be a stack counter in an image stack.<br>
<br>
To account for this, ``rolling_ball`` has a ``kernel`` argument<br>
which allows you to specify the kernel to be used. A kernel must<br>
have the same dimensionality as the image (Note: dimensionality,<br>
not shape). To help with it's creation, two default kernels are<br>
provided by ``skimage``. ``ball_kernel`` specifies a ball shaped<br>
kernel and is used as the default kernel. ``ellipsoid_kernel``<br>
specifies an ellipsoid shaped kernel.

In [ ]:
image = data.coins()
kernel = restoration.ellipsoid_kernel(
    (70.5 * 2, 70.5 * 2),
    70.5 * 2
)

In [ ]:
background = restoration.rolling_ball(
    image,
    kernel=kernel
)
plot_result(image, background)
plt.show()

####################################################################<br>
You can also use ``ellipsoid_kernel`` to recreate the previous,<br>
unexpected result and see that the effective (spatial) filter size<br>
was reduced.

In [ ]:
image = data.coins()

In [ ]:
kernel = restoration.ellipsoid_kernel(
    (10 * 2, 10 * 2),
    255 * 2
)

In [ ]:
background = restoration.rolling_ball(
    image,
    kernel=kernel
)
plot_result(image, background)
plt.show()

####################################################################<br>
Higher Dimensions<br>
-----------------<br>
<br>
Another feature of ``rolling_ball`` is that you can directly<br>
apply it to higher dimensional images, e.g., a z-stack of images<br>
obtained during confocal microscopy. The number of kernel<br>
dimensions must match the image dimensions, hence the kernel shape<br>
is now 3 dimensional.

In [ ]:
image = data.cells3d()[:, 1, ...]
background = restoration.rolling_ball(
    image,
    kernel=restoration.ellipsoid_kernel(
        (1, 21, 21),
        0.1
    )
)

In [ ]:
plot_result(image[30, ...], background[30, ...])
plt.show()

####################################################################<br>
A kernel size of 1 does not filter along this axis. In other words,<br>
above filter is applied to each image in the stack individually.<br>
<br>
However, you can also filter along all 3 dimensions at the same<br>
time by specifying a value other than 1.

In [ ]:
image = data.cells3d()[:, 1, ...]
background = restoration.rolling_ball(
    image,
    kernel=restoration.ellipsoid_kernel(
        (5, 21, 21),
        0.1
    )
)

In [ ]:
plot_result(image[30, ...], background[30, ...])
plt.show()

####################################################################<br>
Another possibility is to filter individual pixels along the<br>
planar axis (z-stack axis).

In [ ]:
image = data.cells3d()[:, 1, ...]
background = restoration.rolling_ball(
    image,
    kernel=restoration.ellipsoid_kernel(
        (100, 1, 1),
        0.1
    )
)

In [ ]:
plot_result(image[30, ...], background[30, ...])
plt.show()